
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example-05.ipynb)

# Example 05: Custom Benchmark Implementation

This example demonstrates how to extend MoeaBench by creating your own vectorized multi-objective problem (MOP), allowing you to optimize custom domains with the same analytical tools.

In [ ]:
!pip install --quiet git+https://github.com/monacofj/moeabench.git


In [ ]:
from MoeaBench import mb
import numpy as np

# 1. Component: Custom Problem Logic
class MyProblem(mb.mops.BaseMop):
    """
    A custom bi-objective problem (ZDT1-like) implemented with NumPy.
    """
    def __init__(self):
        # M=2 Objectives, N=10 Variables, Bounds=[0, 1]
        super().__init__(M=2, N=10, xl=0.0, xu=1.0)

    def evaluation(self, X, n_ieq_constr=0):
        # Evaluation MUST be vectorized (X is a population matrix)
        f1 = X[:, 0]
        g = 1 + 9 * np.sum(X[:, 1:], axis=1) / (self.N - 1)
        f2 = g * (1 - np.sqrt(abs(f1 / g)))
        # Return results in the 'F' key
        return {'F': np.column_stack([f1, f2])}

In [ ]:
# 2. Setup: Instantiate and use the custom benchmark
mop1 = MyProblem()

exp1 = mb.experiment()
exp1.name = "MyCustomProblem"
exp1.mop = mop1
exp1.moea = mb.moeas.NSGA2deap(population=100, generations=50)

# 3. Execution
print(f"Optimizing {exp1.name}...")
exp1.run()

# 4. Visualization (Spatial Perspective)
print("Plotting results...")
mb.view.spaceplot(exp1, title="Final Front of Custom Benchmark")

### Interpretation

Creating custom problems is as simple as defining the `evaluation` function. Because MoeaBench expects a vectorized input (X), you can leverage the full speed of NumPy.

Once defined, your custom mops becomes a "first-class citizen" in the library, compatible with all `mb.view` tools (like `spaceplot`) and statistical measures.